In [1]:
import cv2
# import matplotlib
# import matplotlib.pyplot as plt
import numpy as np
import glob
import torch
# import torch.nn as nn 
import time
import math
# import torch.nn.functional as F
#from net import *
#from utils import *
#from tracker import *
import onnx

from main.paths import ROOT_PATH  # isort:skip
import argparse
import os.path as osp
import sys
import cv2
from loguru import logger

import torch

import random
import os
import numpy as np

import cv2
# import matplotlib
# import matplotlib.pyplot as plt
import numpy as np
import glob
import torch
# import torch.nn as nn 
import time
import math
# import torch.nn.functional as F

from utils import *
from tracker import *
from videoanalyst.config.config import cfg as root_cfg
from videoanalyst.config.config import specify_task
from videoanalyst.data import builder as dataloader_builder
from videoanalyst.engine import builder as engine_builder
from videoanalyst.model import builder as model_builder
from videoanalyst.optim import builder as optim_builder
from videoanalyst.utils import Timer, complete_path_wt_root_in_cfg, ensure_dir
from videoanalyst.pipeline import builder as pipeline_builder
from videoanalyst.engine.builder import build as tester_builder
from videoanalyst.engine.monitor.monitor_impl.tensorboard_logger import TensorboardLogger
from videoanalyst.config.config import cfg as root_cfg
from videoanalyst.config.config import specify_task
from videoanalyst.engine.builder import build as tester_builder
from videoanalyst.model import builder as model_builder
from videoanalyst.pipeline import builder as pipeline_builder
from videoanalyst.utils import complete_path_wt_root_in_cfg

In [2]:
cv2.setNumThreads(1)


In [3]:
parsed_args_config = '/home/meysam/test-apps/STMTrack/experiments/stmtrack/test/got10k/stmtrack-effnet-got.yaml'
exp_cfg_path = osp.realpath(parsed_args_config)

root_cfg.merge_from_file(exp_cfg_path)

root_cfg = complete_path_wt_root_in_cfg(root_cfg, ROOT_PATH)

task, task_cfg = specify_task(root_cfg.test)

task_cfg.freeze()

log_dir = osp.join(task_cfg.exp_save, task_cfg.exp_name, "logs")

ensure_dir(log_dir)

In [4]:

logger.configure(
        handlers=[
            dict(sink=sys.stderr, level="INFO"),
            dict(sink=osp.join(log_dir, "train_log.txt"),
                 enqueue=True,
                 serialize=True,
                 diagnose=True,
                 backtrace=True,
                 level="INFO")
        ],
        extra={"common_to_all": "default"},
    )

# backup config
logger.info("Load experiment configuration at: %s" % exp_cfg_path)
logger.info(
    "Merged with root_cfg imported from videoanalyst.config.config.cfg")
cfg_bak_file = osp.join(log_dir, "%s_bak.yaml" % task_cfg.exp_name)

with open(cfg_bak_file, "w") as f:
    f.write(task_cfg.dump())

logger.info("Task configuration backed up at %s" % cfg_bak_file)


2024-08-26 11:00:06.721 | INFO     | __main__:<module>:15 - Load experiment configuration at: /home/meysam/test-apps/STMTrack/experiments/stmtrack/test/got10k/stmtrack-effnet-got.yaml
2024-08-26 11:00:06.722 | INFO     | __main__:<module>:16 - Merged with root_cfg imported from videoanalyst.config.config.cfg
2024-08-26 11:00:06.740 | INFO     | __main__:<module>:23 - Task configuration backed up at /home/meysam/test-apps/STMTrack/logs/stmtrack-effnet-got-test/got10k/logs/got10k_bak.yaml


In [5]:
model = model_builder.build(task, task_cfg.model)


2024-08-26 11:00:07.585 | INFO     | videoanalyst.model.module_base:update_params:60 - Load pretrained STMTrack parameters from: /home/meysam/test-apps/STMTrack/snapshots/stmtrack-effnet-adam-got-train-val-mdim-16/epoch-10.pkl whose md5sum is eb75584f9834d555c7ef99dde0e7fe30


In [6]:
class Basemodel_Q(nn.Module):
    def __init__(self):
        super(Basemodel_Q, self).__init__()
        self.backbone_q = model.basemodel_q
        self.neck_q = model.neck_q
    def forward(self, search_img):
        fq = self.backbone_q(search_img)
        fq = self.neck_q(fq)
        return fq

In [7]:
net = Basemodel_Q()
ONNX_FILE_PATH = "onnx-files/mdim-16/backbone_q.onnx"


In [8]:
inp = torch.randn(1,3,200,200).cuda()
net.eval()
net.cuda()
y = net(inp)

In [9]:
torch.onnx.export(net, inp, ONNX_FILE_PATH, input_names=["search_img"], output_names=["fq"], export_params=True)

In [10]:

class Memorize(nn.Module):
    def __init__(self):
        super(Memorize, self).__init__()
        self.backbone_m = model.basemodel_m
        self.neck_m = model.neck_m
    def forward(self, im_crop, fg_bg_label_map):
        fm = self.backbone_m(im_crop, fg_bg_label_map)
        fm = self.neck_m(fm)
        fm = fm.permute(1, 0, 2, 3).unsqueeze(0).contiguous()  # B, C, T, H, W
        return fm


In [11]:
net = Memorize()#(model.basemodel_m,model.neck_m)

input = torch.ones(1,3,200,200).cuda()
fg_bg = torch.ones(1,1,200,200).cuda()
net.eval()
net.cuda()
fm = net(input,fg_bg)


In [12]:
fm.shape

torch.Size([1, 20, 1, 25, 25])

In [13]:
ONNX_FILE_PATH = "onnx-files/mdim-16/memorize.onnx"
input = torch.randn(1,3,200,200).cuda()
fg_bg = torch.randn(1,1,200,200).cuda()
net.eval()
net.cuda()
torch.onnx.export(net, (input,fg_bg), ONNX_FILE_PATH, input_names=["img","fg_bg_label_map"], output_names=["fm"], export_params=True)


In [14]:
hps = dict(
        total_stride=8,
        score_size=25,
        score_offset=-1,
        test_lr=0.95,
        penalty_k=0.04,
        window_influence=0.21,
        windowing="cosine",
        m_size=200,
        q_size=200,
        min_w=10,
        min_h=10,
        phase_memorize="memorize",
        phase_track="track",
        corr_fea_output=False,
        num_segments=4,
        confidence_threshold=0.6,
        gpu_memory_threshold=-1,
        search_area_factor=4.0,
        visualization=False )

In [15]:
def get_box(xy_ctr, offsets):
    offsets = offsets.permute(0, 2, 3, 1)  # (B, H, W, C), C=4
    offsets = offsets.reshape(offsets.shape[0], -1, 4)
    xy0 = (xy_ctr[:, :, :] - offsets[:, :, :2])
    xy1 = (xy_ctr[:, :, :] + offsets[:, :, 2:])
    bboxes_pred = torch.cat([xy0, xy1], 2)

    return bboxes_pred

In [16]:
class Head(nn.Module):
    def __init__(self,head):
        super(Head, self).__init__()
        self.head = head
        self.total_stride = 8.0
        score_offset = (hps["q_size"] - 1.0 - (hps["score_size"] - 1) * hps["total_stride"]) // 2.0
        self.fm_ctr = get_xy_ctr_np(hps["score_size"], score_offset, hps["total_stride"])
    def forward(self,fm1,fm2,fm3,fm4,fm5,fm6,fq):

        fm = torch.cat([fm1,fm2,fm3,fm4,fm5,fm6],dim = 2)
        y = self.head.memory_read(fm, fq)
        cls_score, ctr_score, offsets, cls_feat = self.head.solve(y)
        
        cls_score = cls_score.permute(0, 2, 3, 1)
        cls_score = cls_score.reshape(cls_score.shape[0], -1, 1)

        ctr_score = ctr_score.permute(0, 2, 3, 1)
        ctr_score = ctr_score.reshape(ctr_score.shape[0], -1, 1)

        offsets = torch.exp(self.head.si * offsets + self.head.bi) * 8.0

        
        fm_ctr = self.fm_ctr.to(offsets.device)
        bbox = get_box(fm_ctr, offsets)
        fcos_cls_score_final = cls_score
        fcos_ctr_score_final = ctr_score
        fcos_bbox_final = bbox

        
        fcos_cls_prob_final = torch.sigmoid(fcos_cls_score_final)
        fcos_ctr_prob_final = torch.sigmoid(fcos_ctr_score_final)
        # apply centerness correction
        fcos_score_final = fcos_cls_prob_final * fcos_ctr_prob_final
        return fcos_score_final, fcos_bbox_final


In [17]:

net = Head(model.head)
net.eval()
net.cuda()
ONNX_FILE_PATH = "onnx-files/mdim-16/head.onnx"
# fm = torch.randn(1, 512, 6, 25, 25).cuda()
fm1 = torch.randn(1, 20, 1, 25, 25).cuda()
fm2 = torch.randn(1, 20, 1, 25, 25).cuda()
fm3 = torch.randn(1, 20, 1, 25, 25).cuda()
fm4 = torch.randn(1, 20, 1, 25, 25).cuda()
fm5 = torch.randn(1, 20, 1, 25, 25).cuda()
fm6 = torch.randn(1, 20, 1, 25, 25).cuda()

fq = torch.randn(1, 20, 25, 25).cuda()

In [18]:
score, bbox = net(fm1,fm2,fm3,fm4,fm5,fm6,fq)

In [19]:
torch.onnx.export(net, (fm1,fm2,fm3,fm4,fm5,fm6,fq), 
                  ONNX_FILE_PATH, input_names=["fm1","fm2","fm3","fm4","fm5","fm6","fq"], 
                  output_names=["score", "bbox"]
                  , export_params=True)

/home/meysam/test-apps/STMTrack/videoanalyst/model/task_head/taskhead_impl/stm_head.py:95: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  w = torch.bmm(fm, fq) / math.sqrt(C)  # B, THW, HW
